## Try calling gemini for code reviewing

In [1]:
import os
from dotenv import load_dotenv
from google import genai

In [2]:
load_dotenv()
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
model_name = "gemini-2.5-flash"

Prompt engineering

In [5]:
sample_code = """
def add(a, b):
    return a - b
"""

In [6]:
prompt = f"""
You are a senior software engineer acting as a static code reviewer.

OBJECTIVE:
Analyze the given code and report ONLY findings that are directly supported by the code itself.

DEFINITIONS:
- Bug: A logic, syntax, or runtime error that will cause incorrect behavior.
- Risk: A realistic scenario where this code could fail or be misleading in real usage.
- Suggestion: A concrete improvement that improves correctness or clarity.

SUMMARY RULES:
- Provide a short, factual overall assessment of the code.
- Do NOT include praise or vague statements.
- Base the summary strictly on the findings below.
- If the code is incorrect, state that clearly.

STRICT RULES:
- Do NOT speculate beyond the given code.
- Do NOT invent context or usage.
- If an issue cannot be proven from the code, do NOT include it.
- If no issues exist for a category, return an empty array.

OUTPUT FORMAT:
Return ONLY valid JSON that strictly follows this schema:

{{
  "summary": {{
    "overall_assessment": string,
    "confidence": "high | medium | low"
  }},
  "bugs": [
    {{
      "line": number,
      "type": "logic | syntax | runtime",
      "description": string
    }}
  ],
  "risks": [
    {{
      "description": string
    }}
  ],
  "suggestions": [
    {{
      "description": string
    }}
  ]
}}

CODE:
{sample_code}
"""


In [7]:
try: 
    response = client.models.generate_content(model=model_name, contents=prompt) 
    print(response.text) 
except Exception as e: 
    print(f"An error occurred: {e}")

```json
{
  "summary": {
    "overall_assessment": "The code contains a critical logic error where the function's name contradicts its implemented behavior.",
    "confidence": "high"
  },
  "bugs": [
    {
      "line": 2,
      "type": "logic",
      "description": "The function is named 'add' but performs subtraction (a - b). This leads to incorrect results if callers expect addition."
    }
  ],
  "risks": [
    {
      "description": "Any module or component that calls this function expecting an addition operation will receive an incorrect subtracted value, leading to silent calculation errors throughout the system."
    }
  ],
  "suggestions": [
    {
      "description": "Rename the function to 'subtract' to accurately reflect its operation, or change the operation to 'a + b' to align with the 'add' function name."
    }
  ]
}
```
